In [3]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score
from train_set import Training_set
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import pandas as pd
from numpy import std, mean

In [25]:
bio_features = pd.read_csv("bio_data.csv", index_col= 0)
X = bio_features
#Y has the last col with labels
y = Training_set.iloc[:, -1]
# split the dataset to train and test, the 2nd is for the non feature selection
Training_data = pd.merge(X, y, left_index=True, right_index=True)
print(Training_data)

X = Training_data.iloc[:, :-1]
#Y has the last col with labels
y = Training_data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X)

           69274433-70345552  196908262-196937230  19049822-20716660  \
Array.129                 -1                    0                  0   
Array.34                   0                    0                  0   
Array.67                   0                   -1                  0   
Array.24                  -1                    0                  0   
Array.22                   0                    0                  1   
...                      ...                  ...                ...   
Array.10                   0                    0                  0   
Array.123                  0                    1                  1   
Array.100                 -1                    0                  0   
Array.134                  0                    1                  1   
Array.130                 -1                    0                  0   

           131304839-132036965  70574871-71644041  97852156-98629015  \
Array.129                    1                  1              

In [23]:
# Select the most important features
X_train_selected = X_train
X_test_selected = X_test

# Normalize the features
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train_selected)
X_test_norm = scaler.transform(X_test_selected)

# Nested CV

In [7]:
# Split the generated data into training and testing sets
skf_outer = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
actual_scores = []

# Define the parameter grid for the random forest classifier
param_grid = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']}

for train_index_inner, test_index_inner in skf_outer.split(X_train_norm, y_train):
    X_train_inner, X_test_inner = X_train_norm[train_index_inner], X_train_norm[test_index_inner]
    y_train_inner, y_test_inner = y_train.iloc[train_index_inner], y_train.iloc[test_index_inner]

    # parameter tuning for the random forest classifier using GridSearchCV
    knn = KNeighborsClassifier()

    # Inner cross-validation loop for hyperparameter tuning
    skf_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    # defining parameter range
    grid = GridSearchCV(knn, param_grid, cv=skf_inner, scoring='accuracy', return_train_score=False,verbose=1)

    grid_search=grid.fit(X_train_inner, y_train_inner)
    best_model = grid_search.best_estimator_
    yhat = best_model.predict(X_test_inner)
    acc= accuracy_score(y_test_inner,yhat)
    actual_scores.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, grid_search.best_score_, grid_search.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (mean(actual_scores), std(actual_scores)))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
>acc=0.875, est=0.860, cfg={'n_neighbors': 7, 'weights': 'distance'}
Fitting 5 folds for each of 8 candidates, totalling 40 fits
>acc=0.875, est=0.828, cfg={'n_neighbors': 3, 'weights': 'uniform'}
Fitting 5 folds for each of 8 candidates, totalling 40 fits
>acc=0.938, est=0.812, cfg={'n_neighbors': 3, 'weights': 'distance'}
Fitting 5 folds for each of 8 candidates, totalling 40 fits
>acc=0.875, est=0.876, cfg={'n_neighbors': 3, 'weights': 'uniform'}
Fitting 5 folds for each of 8 candidates, totalling 40 fits
>acc=0.875, est=0.844, cfg={'n_neighbors': 3, 'weights': 'distance'}
Accuracy: 0.887 (0.025)


# Here we do a nestedcv but this time picking each of the outputs from the previous gridsearch of each loop and seeing each performance in all the loops picking the top one for the final model

In [19]:
# Split the generated data into training and testing sets
skf_outer2 = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
actual_scores2 = []
for train_index_inner, test_index_inner in skf_outer2.split(X_train, y_train):
    X_train_inner, X_test_inner = X_train.iloc[train_index_inner], X_train.iloc[test_index_inner]
    y_train_inner, y_test_inner = y_train.iloc[train_index_inner], y_train.iloc[test_index_inner]
    knn_model = KNeighborsClassifier(n_neighbors= 3, weights= 'distance')

    knn_eval = knn_model.fit(X_train_inner, y_train_inner)

    yhat2 = knn_eval.predict(X_test_inner)
    acc2= accuracy_score(y_test_inner,yhat2)
    actual_scores2.append(acc2)
    # report progress
    print('>acc=%.3f' % (acc2))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (mean(actual_scores2), std(actual_scores2)))

>acc=0.812
>acc=0.875
>acc=0.875
>acc=0.938
>acc=0.938
Accuracy: 0.887 (0.047)


# After finding optimal hyperparameters test Performance on the Test set

In [20]:
#  Put a number of k and uniform based on the previous part
knn_final = KNeighborsClassifier(n_neighbors=3, weights='distance')

knn_final.fit(X_train_norm, y_train)

y_test_hat = knn_final.predict(X_test_norm)

test_accuracy = accuracy_score(y_test, y_test_hat) * 100

print("Accuracy for our testing dataset with tuning is : {:.2f}%".format(test_accuracy))

precision = metrics.precision_score(y_test, y_test_hat, average='macro')
recall = metrics.recall_score(y_test, y_test_hat, average='macro')
f1_score = metrics.f1_score(y_test, y_test_hat, average='macro')

print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1-score: {f1_score:.3f}')

Accuracy for our testing dataset with tuning is : 85.00%
Precision: 0.900
Recall: 0.833
F1-score: 0.830


In [24]:
# Create a classification report
from sklearn.metrics import classification_report


cr = classification_report(y_test, y_test_hat)
print(cr)

              precision    recall  f1-score   support

       HER2+       1.00      1.00      1.00         7
         HR+       0.70      1.00      0.82         7
  Triple Neg       1.00      0.50      0.67         6

    accuracy                           0.85        20
   macro avg       0.90      0.83      0.83        20
weighted avg       0.89      0.85      0.84        20



In [ ]:
import pickle
pickle.dump(knn_final, open('knn_bio.pkl', 'wb'))